# Exploring Data

In [7]:
import pandas as pd

In [12]:
unchanged_df = pd.read_csv('./data/kc_house_data.csv')

unchanged_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [13]:
#an outside consultant has suggested potentially dropping these features. 
#this is a brief investigation into what the data looks like after doing so.

#suggested_drop = ['id', 'date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
#suggested_df = unchanged_df.drop(suggested_drop, axis=1)

#suggested_df.head()

### General Observations

We want to look at the different features in this dataset, and first use a bit of common sense to remove unnecessary pieces of data.

## Column Names and Descriptions for King County Data Set
* `id` - Unique identifier for a house
* `date` - Date house was sold
* `price` - Sale price (prediction target)
* `bedrooms` - Number of bedrooms
* `bathrooms` - Number of bathrooms
* `sqft_living` - Square footage of living space in the home
* `sqft_lot` - Square footage of the lot
* `floors` - Number of floors (levels) in house
* `waterfront` - Whether the house is on a waterfront
  * Includes Duwamish, Elliott Bay, Puget Sound, Lake Union, Ship Canal, Lake Washington, Lake Sammamish, other lake, and river/slough waterfronts
* `view` - Quality of view from house
  * Includes views of Mt. Rainier, Olympics, Cascades, Territorial, Seattle Skyline, Puget Sound, Lake Washington, Lake Sammamish, small lake / river / creek, and other
* `condition` - How good the overall condition of the house is. Related to maintenance of house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each condition code
* `grade` - Overall grade of the house. Related to the construction and design of the house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each building grade code
* `sqft_above` - Square footage of house apart from basement
* `sqft_basement` - Square footage of the basement
* `yr_built` - Year when house was built
* `yr_renovated` - Year when house was renovated
* `zipcode` - ZIP Code used by the United States Postal Service
* `lat` - Latitude coordinate
* `long` - Longitude coordinate
* `sqft_living15` - The square footage of interior housing living space for the nearest 15 neighbors
* `sqft_lot15` - The square footage of the land lots of the nearest 15 neighbors


Some of these variables won't be useful in helping determining house value. While things like latitude, longitude, and zipcode could certainly point to different neighborhoods being of higher value, this analysis is all taking place within King County, and we don't want to train our model to mistakenly associate coordinates of a home with value,

Sqft_living15 and sqft_lot15 can introduce far too many random occurences, as it takes into account the "nearest 15 neighbors", so it too will be removed. Similarly, sqft_basement and sqft_above simply divide up the total square footage of a house, which is already appropriately represented by sqft_living. So we will remove these two sub-variables as well.

In [15]:
drop = ['id', 'lat', 'long', 'zipcode', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'sqft_basement']
adjusted_df = unchanged_df.drop(drop, axis=1)

We should also look at what range of time the data was collected over, to decide if the date column is of any use.

In [17]:
adjusted_df['date'] = pd.to_datetime(unchanged_df['date'])
adjusted_df.sort_values('date', ascending=True)[:1]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated
7316,2014-05-02,435000.0,4,1.0,1450,8800,1.0,NO,NONE,Good,7 Average,1954,0.0


In [19]:
adjusted_df.sort_values('date', ascending=False)[:1]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated
16580,2015-05-27,1310000.0,4,2.25,3750,5000,2.0,NO,NONE,Very Good,8 Good,1924,0.0


This data is only collected over a single year, so the date column (which indicates when each house was sold) can be removed since there will be no significant impact of inflation or cultural trends in housing.

In [20]:
drop_date = ['date']
adjusted_df.drop(drop_date, axis=1, inplace=True)

## Data Cleaning
### Null Values


Next we need to check for any null values in our data, and handle them appropriately.

In [21]:
adjusted_df.isna().sum()

price              0
bedrooms           0
bathrooms          0
sqft_living        0
sqft_lot           0
floors             0
waterfront      2376
view              63
condition          0
grade              0
yr_built           0
yr_renovated    3842
dtype: int64

The first and simplest category that needs cleaning is the 'view' data. Only 63 entries of over 21,000 have no data on view, so we drop these entries entirely.

In [22]:
cleaned_df = adjusted_df.dropna(subset=['view'])
cleaned_df.isna().sum()

price              0
bedrooms           0
bathrooms          0
sqft_living        0
sqft_lot           0
floors             0
waterfront      2370
view               0
condition          0
grade              0
yr_built           0
yr_renovated    3830
dtype: int64

The next highest collection of null values is found in the waterfront column. It's important at this point to look at what sort of values are present in this column.

In [23]:
cleaned_df['waterfront'].value_counts(dropna=False)

NO     19019
NaN     2370
YES      145
Name: waterfront, dtype: int64

The vast majority of these entries are listed as not being waterfront property, but since we dont want to drop such a large number of entries just because of missing values, we will interpret Nan to be equivalent to "NO".

In [24]:
cleaned_df['waterfront'].fillna("NO", inplace=True)
cleaned_df['waterfront'].value_counts(dropna=False)

C:\Users\shade\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


NO     21389
YES      145
Name: waterfront, dtype: int64

Next, we have to address the largest set of null values, the ones in yr_renovated. almost 4,000 of our 21,000 is a lot to drop.

We could change the values to 0, but that could affect our data by treating a large portion of our data as if they were renovated thousands of years ago, and we could accidentally train our model to look for a pattern among ancient homes.

Instead, we will look at what sort of values are present in the yr_renovated column.

In [25]:
#pd.set_option("display.max_rows", None)
cleaned_df['yr_renovated'].value_counts(dropna=False)

0.0       16961
NaN        3830
2014.0       73
2003.0       31
2013.0       31
          ...  
1944.0        1
1948.0        1
1976.0        1
1934.0        1
1953.0        1
Name: yr_renovated, Length: 71, dtype: int64

Most renovations happened within recent decades, which makes sense. The dataset most likely will not benefit from tracking renovations from too long ago, either. So we will replace yr_renovated with a new column which indicates if a home has been renovated since the year 2000. 

In [26]:
new_df['renovated_2000'] = new_df['yr_renovated'].map(lambda x: 1 if x>=2000 else 0)
new_df.drop('yr_renovated', axis=1, inplace=True)
new_df

<ipython-input-26-2f2ee647705b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['renovated_2000'] = new_df['yr_renovated'].map(lambda x: 1 if x>=2000 else 0)
C:\Users\shade\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,renovated_2000
0,221900.0,3,1.00,1180,5650,1.0,NO,NONE,Average,7 Average,1955,0
1,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,1951,0
2,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,1933,0
3,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1965,0
4,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1987,0
...,...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,3,2.50,1530,1131,3.0,NO,NONE,Average,8 Good,2009,0
21593,400000.0,4,2.50,2310,5813,2.0,NO,NONE,Average,8 Good,2014,0
21594,402101.0,2,0.75,1020,1350,2.0,NO,NONE,Average,7 Average,2009,0
21595,400000.0,3,2.50,1600,2388,2.0,NO,NONE,Average,8 Good,2004,0


In [27]:
cleaned_df['renovated_2000'].value_counts()

#not a high number of renovated homes, but we'll keep this around for now 
#in order to identify how much of a difference renovations make in price.

0    21155
1      379
Name: renovated_2000, dtype: int64

In [28]:
#checking to make sure our null values are taken care of
cleaned_df.isna().sum()

price             0
bedrooms          0
bathrooms         0
sqft_living       0
sqft_lot          0
floors            0
waterfront        0
view              0
condition         0
grade             0
yr_built          0
renovated_2000    0
dtype: int64

No more null values!

## Next Steps
Now we can use our cleaned data in the main notebook to create predictive models:

[Regression Analysis Notebook!](./kc_linear_regression_analysis.ipynb)